In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('default_of_credit_card_clients__courseware_version_1_21_19.xls')

In [3]:
df.shape

(30000, 25)

In [4]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object')

In [5]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [6]:
df.shape

(30000, 25)

In [14]:
df['ID'].nunique() #duplicates somewhere

29687

## Locate the duplicates

In [10]:
id_counts = df['ID'].value_counts()
id_counts.head()

4e2380e6-a8cf    2
63b20368-1a04    2
c69162db-4864    2
f7fe4368-18e7    2
2618fdc4-e731    2
Name: ID, dtype: int64

In [11]:
id_counts.value_counts() #confirming no IDs occur more than twice. 

1    29374
2      313
Name: ID, dtype: int64

In [12]:
dupe_mask = id_counts == 2 #bool or logical mask to locate duplicates
dupe_mask[0:5]

4e2380e6-a8cf    True
63b20368-1a04    True
c69162db-4864    True
f7fe4368-18e7    True
2618fdc4-e731    True
Name: ID, dtype: bool

In [15]:
id_counts.index[0:5] #duplicated IDs selected

Index(['4e2380e6-a8cf', '63b20368-1a04', 'c69162db-4864', 'f7fe4368-18e7',
       '2618fdc4-e731'],
      dtype='object')

In [16]:
dupe_ids = id_counts.index[dupe_mask] #store duplicates

In [18]:
dupe_ids = list(dupe_ids) #convert to list
len(dupe_ids)             #length

313

In [19]:
dupe_ids[0:5] #verify the data

['4e2380e6-a8cf',
 '63b20368-1a04',
 'c69162db-4864',
 'f7fe4368-18e7',
 '2618fdc4-e731']

In [24]:
#to look at values of features to see if anything amiss.. 
#isin(dupe_ids[0:3] is another logical mask to grab rows with the dupe IDs
df.loc[df['ID'].isin(dupe_ids[0:3]),:].head(10) 

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
9116,63b20368-1a04,200000,1,3,1,29,-1,-1,-1,-1,...,8335,971,4278,46722,22985,8346,971,4296,21767,0
9216,63b20368-1a04,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20410,c69162db-4864,130000,1,2,2,25,0,0,0,0,...,125300,4125,4210,31200,4268,89527,107,500,1000,0
20510,c69162db-4864,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27339,4e2380e6-a8cf,170000,1,2,1,37,Not available,2,0,0,...,82034,85664,89272,0,5000,5000,5000,5000,5000,0
27439,4e2380e6-a8cf,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We can see some duplicates here, and it looks like every duplicate ID has one row with data, and another row with all zeros. Is this the case for every duplicate ID? Let's check.

In [25]:
df_zero_mask = df == 0 #create Bool matrix of entire DataFrame

In [29]:
feature_zero_mask = df_zero_mask.iloc[:, 1:].all(axis=1) 
# .all(axis=1) which will return True if and only if every column 
# in that row is True. 

The goal: create a Boolean series, feature_zero_mask, that identifies every row where all the elements starting from the second column (the features and response, but not the IDs) are 0. 

In [35]:
sum(feature_zero_mask) #zero rows = duplicate IDs count

315

In [37]:
df_clean_1 = df.loc[~feature_zero_mask, :].copy()
# we used the logical not operator, ~, to select all the rows that 
# don't have zeros 

In [38]:
df_clean_1.shape

(29685, 25)

In [39]:
df_clean_1['ID'].nunique()

29685

duplicates dropped successfully. 